# Simple RDataFrame example

RDataFrame allows to perform operations efficiently in two ways:
* get multiple results in one shot
* multithreading 

Let's first take a look at the dataset

In [ ]:
import ROOT
import uproot4

In [ ]:
f = uproot4.open('data/tnp1.root')
t = f['Data_13TeV_All']

t.show()
print('\nEntries: {}'.format(t.num_entries))

### Optimal (event loop run once)

In [ ]:
rdf = ROOT.RDataFrame('Data_13TeV_All', 'data/tnp1.root')
names = [name for name in rdf.GetColumnNames()]

In [ ]:
%%time 

ptrs = [rdf.Histo1D(name) for name in names]

histos = []
for ptr in ptrs:
    histos.append(ptr.GetValue())
    
print('Event loop run {} time(s)'.format(rdf.GetNRuns()))

### Suboptimal (event loop run once per branch, TTree::Draw-style)

In [ ]:
rdf = ROOT.RDataFrame('Data_13TeV_All', 'data/tnp1.root')
names = [name for name in rdf.GetColumnNames()]

In [ ]:
%%time

histos = []
for name in names:
    ptr = rdf.Histo1D(name)
    histos.append(ptr.GetValue())
    
print('Event loop run {} time(s)'.format(rdf.GetNRuns()))

### Optimal, with multithreading

In [ ]:
ROOT.EnableImplicitMT()
rdf = ROOT.RDataFrame('Data_13TeV_All', 'data/tnp1.root')
names = [name for name in rdf.GetColumnNames()]

In [ ]:
%%time 

ptrs = [rdf.Histo1D(name) for name in names]

histos = []
for ptr in ptrs:
    histos.append(ptr.GetValue())

### If we want to see the histograms (ROOT Jupyter trick)

In [ ]:
canvases = []

for histo in histos:
    canvas = ROOT.TCanvas(histo.GetName(), histo.GetName())
    histo.Draw()
    canvases.append(canvas)

for canvas in canvases:
    canvas.Draw()

### If we want to see the histograms (matplotlib)

In [ ]:
import matplotlib.pyplot as plt
import boost_histogram as bh
import aghast
import mplhep

plt.style.use(mplhep.style.CMS)

def plot(root_histo):
    ghastly_histo = aghast.from_root(root_histo)
    data, edges = aghast.to_numpy(ghastly_histo)

    h = bh.Histogram(bh.axis.Variable(edges[1:-1]))
    h[:] = data
    
    plt.bar(h.axes[0].centers, h.view(), width=h.axes[0].widths)
    plt.show()
    
for histo in histos:
    plot(histo)